<img style="float: left;;" src='Figures/alinco.png' /></a>
    
# <center> <font color= #000047> Actividad 4: Programación lineal ( Ejercicios )</font>
___



### 1. Diseño de la Dieta Óptima

Se quiere producir comida para gatos de la manera más barata, no obstante se debe también asegurar que se cumplan los datos requeridos de analisis nutricional. Por lo que se quiere variar la cantidad de cada ingrediente para cumplir con los estandares nutricionales. Los requisitos que se tienen es que en 100 gramos, se deben tener por lo menos 8 gramos de proteína y 6 gramos de grasa. Así mismo, no se debe tener más de 2 gramos de fibra y 0.4 gramos de sal.  

Los datos nutricionales se pueden obtener de la siguiente tabla:

|Ingrediente|Proteína|Grasa|Fibra|Sal|
|----|---- |---- |---- |---- |
|Pollo|  10.0%|08.0%|00.1%|00.2%
|Carne|  20.0%|10.0%|00.5%|00.5%
|Cordero|15.0%|11.0%|00.5%|00.7%
|Arroz|  00.0%|01.0%|10.0%|00.2%
|Trigo|  04.0%|01.0%|15.0%|00.8%
|Gel|    00.0%|00.0%|00.0%|00.0%

Los costos de cada producto son:

Ingrediente|Costo por gramo
:----|----
Pollo|$\$$0.013
Carne|$\$$0.008
Cordero|$\$$0.010
Arroz|$\$$0.002
Trigo|$\$$0.005
Gel|$\$$0.001    

Lo que se busca optimizar en este caso es la cantidad de productos que se debe utilizar en la comida de gato, para simplificar la notación se van a nombrar las siguientes variables: 

$x_1:$ Gramos de pollo  
$x_2:$ Gramos de carne  
$x_3:$ Gramos de cordero  
$x_4:$ Gramos de arroz  
$x_5:$ Gramos de trigo  
$x_6:$ Gramos de gel  

Con los datos, se puede plantear la función objetivo, está dada por la siguiente expresión:

$$\min 0.013 x_1 + 0.008 x_2 + 0.010 x_3 + 0.002 x_4 + 0.005 x_5 + 0.001 x_6$$

Las restricciones estarían dadas por el siguiente conjunto de ecuaciones:

$x_1+x_2+x_3+x_4+x_5+x_6=100$  

$(10.0 x_1+ 20.0 x_2+ 15.0 x_3+ 00.0 x_4+ 04.0 x_5+ 00.0 x_6)/100 \geq 8.0$  

$(08.0 x_1+ 10.0 x_2+ 11.0 x_3+ 01.0 x_4+ 01.0 x_5+ 00.0 x_6)/100 \geq 6.0$  

$(00.1 x_1+ 00.5 x_2+ 00.5 x_3+ 10.0 x_4+ 15.0 x_5+ 00.0 x_6)/100 \leq 2.0$  

$(00.2 x_1+ 00.5 x_2+ 00.7 x_3+ 00.2 x_4+ 00.8 x_5+ 00.0 x_6)/100 \leq 0.4$  

La primer condición asegura que la cantidad de productos que se usará cumple con los 100 gramos. Las siguientes sólo siguen los lineamientos planteados para cumplir con los requisitos nutrimentales. 

In [ ]:
import numpy as np
from scipy import optimize as opt

# costos
c = np.array([0.013, 0.008, 0.010, 0.002, 0.005, 0.001])

A_ineq = np.array([[1, 1, 1, 1, 1, 1]])
b_ineq = np.array([100])

# Restricciones
A_ub = np.array([
    [-0.10, -0.20, -0.15,  0.00, -0.04,  0.00],   # Proteina <= -8
    [-0.08, -0.10, -0.11, -0.01, -0.01,  0.00],   # Grasa <= -6
    [ 0.001, 0.005, 0.005, 0.10,  0.15,  0.00],   # Fibra <= 2
    [ 0.002, 0.005, 0.007, 0.002, 0.008, 0.00]    # Sal <= 0.4
])
b_ub = np.array([-8, -6, 2, 0.4])


bounds = [(0, None)] * 6

solucion = opt.linprog(c, A_ub=A_ub, b_ub=b_ub, A_eq=A_ineq, b_eq=b_ineq,
              bounds=bounds, method="highs")

print("Costo mínimo:", solucion.fun)
print("Gramos de cada ingrediente [pollo, carne, cordero, arroz, trigo, gel]:")
print(solucion.x)
print("Algunos materiales no se utilizaron")


Costo mínimo: 0.52
Gramos de cada ingrediente [pollo, carne, cordero, arroz, trigo, gel]:
[ 0. 60.  0.  0.  0. 40.]


In [66]:
print(f"{"=="*5} Con la resstricción que tenga minimo 1 gramo de cada ingrediente {"=="*10}")
bounds = [(1, None)] * 6

solucion = opt.linprog(c, A_ub=A_ub, b_ub=b_ub, A_eq=A_ineq, b_eq=b_ineq,
              bounds=bounds, method="highs")

print("Costo mínimo:", solucion.fun)
print("Gramos de cada ingrediente [pollo, carne, cordero, arroz, trigo, gel]:")
print(solucion.x)

========== Con la resstricción que tenga minimo 1 gramo de cada ingrediente ====================
Costo mínimo: 0.5313
Gramos de cada ingrediente [pollo, carne, cordero, arroz, trigo, gel]:
[ 1.  57.9  1.   1.   1.  38.1]


### 2. Otro problema de transporte

Supongamos que tenemos que enviar cajas de cervezas de 2 cervecerías (Modelo y Cuauhtémoc Moctezuma) a 5 bares de acuerdo al siguiente gráfico:

<img style="float: center; margin: 0px 0px 15px 15px;" src="https://relopezbriega.github.io/images/Trans_problem.png" width="500px" height="150px" />

Asimismo, supongamos que nuestro gerente financiero nos informa que el costo de transporte por caja de cada ruta se conforma de acuerdo a la siguiente tabla:

In [1]:
#Crear dataframe
import pandas as pd
info = pd.DataFrame({'Bar1': [2, 3], 'Bar2': [4, 1], 'Bar3': [5, 3], 'Bar4': [2, 2], 'Bar5': [1, 3]},
                     index = ['CerveceriaA', 'CerveceriaB'])
info

,Bar1,Bar2,Bar3,Bar4,Bar5
CerveceriaA,2,4,5,2,1
CerveceriaB,3,1,3,2,3


Y por último, las restricciones del problema, van a estar dadas por las capacidades de oferta y demanda de cada cervecería (en cajas de cerveza) y cada bar, las cuales se detallan en el gráfico de más arriba.

Sean:
- $x_i$ cajas transportadas de la cervecería A al Bar $i$,
- $x_{i+5}$ cajas transportadas de la cervecería B al Bar $i$.

Plantear el problema de minimizar el costo de transporte de la forma vista y resolverlo con `linprog`. 



In [59]:
import numpy as np
from scipy import optimize as opt

print(f"{"=="*10} Restricciones de consumo {"=="*10}")
print("xA1 + xB1 = 500")
print("xA2 + xB2 = 900")
print("xA3 + xB3 = 1800")
print("xA4 + xB4 = 200")
print("xA5 + xB5 = 700")

A_ineq = np.array([
    [1,0,0,0,0, 1,0,0,0,0],
    [0,1,0,0,0, 0,1,0,0,0],
    [0,0,1,0,0, 0,0,1,0,0],
    [0,0,0,1,0, 0,0,0,1,0],
    [0,0,0,0,1, 0,0,0,0,1],
])

b_ineq = np.array([500,900,1800,200,700])

print(f"{"=="*10} Restricciones de producción {"=="*10}")
print("xA1 + xA2 + xA3 + xA4 + xA5 ≤ 1000")
print("xB1 + xB2 + xB3 + xB4 + xB5 ≤ 4000")
A_ub = np.array([
    [1,1,1,1,1, 0,0,0,0,0],
    [0,0,0,0,0, 1,1,1,1,1],
])

print(f"{"=="*10} Función objetivo {"=="*10}")
print("min = 2xA1 + 4xA2 + 5xA3 + 2xA4 + 1xA5 + 3xB1 + 1xB2 + 3xB3 + 2xB4 + 3xB5")
c = np.array([2,4,5,2,1, 3,1,3,2,3])

b_ub = np.array([1000,4000])



==================== Restricciones de consumo ====================
xA1 + xB1 = 500
xA2 + xB2 = 900
xA3 + xB3 = 1800
xA4 + xB4 = 200
xA5 + xB5 = 700
==================== Restricciones de producción ====================
xA1 + xA2 + xA3 + xA4 + xA5 ≤ 1000
xB1 + xB2 + xB3 + xB4 + xB5 ≤ 4000
==================== Función objetivo ====================
min = 2xA1 + 4xA2 + 5xA3 + 2xA4 + 1xA5 + 3xB1 + 1xB2 + 3xB3 + 2xB4 + 3xB5


In [60]:
bnds = ((0,None),)*10

solucion = opt.linprog(c, A_ub=A_ub, b_ub=b_ub, A_eq=A_ineq, b_eq=b_ineq, bounds=bnds)
solucion

        message: Optimization terminated successfully. (HiGHS Status 7: Optimal)
        success: True
         status: 0
            fun: 8600.0
              x: [ 3.000e+02  0.000e+00  0.000e+00  0.000e+00  7.000e+02
                   2.000e+02  9.000e+02  1.800e+03  2.000e+02  0.000e+00]
            nit: 1
          lower:  residual: [ 3.000e+02  0.000e+00  0.000e+00  0.000e+00
                              7.000e+02  2.000e+02  9.000e+02  1.800e+03
                              2.000e+02  0.000e+00]
                 marginals: [ 0.000e+00  4.000e+00  3.000e+00  1.000e+00
                              0.000e+00  0.000e+00  0.000e+00  0.000e+00
                              0.000e+00  1.000e+00]
          upper:  residual: [       inf        inf        inf        inf
                                    inf        inf        inf        inf
                                    inf        inf]
                 marginals: [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00
                     

In [61]:
x_min = solucion.x
np.round(x_min)

array([ 300.,    0.,    0.,    0.,  700.,  200.,  900., 1800.,  200.,
          0.])

In [62]:
c.T.dot(x_min)
envios=solucion.x.reshape(2, 5)
print(f"La producción total es de {b_ub.sum()} y la demanda es de {solucion.x.sum()}, existe un exedente de producción")
print("Costo mínimo:", solucion.fun)
print("Envios de A y B a cada bar:")
print(f"A:{envios[0]}")
print(f"B:{envios[1]}")


La producción total es de 5000 y la demanda es de 4100.0, existe un exedente de producción
Costo mínimo: 8600.0
Envios de A y B a cada bar:
A:[300.   0.   0.   0. 700.]
B:[ 200.  900. 1800.  200.    0.]
